In [210]:
%matplotlib  inline 
import pandas as pd
import numpy as np
from igraph  import Graph as IGraph
def getGraph(GV,flag=False): 
    '''图的构件，输入的是numpy的数组，如果3表示图是带权重的，否则没有权重
    '''
    
    if GV.shape[1]==3:
        g=IGraph.TupleList(GV,directed=flag,vertex_name_attr='name',weights=True)  ## 构建图
    else:
        g=IGraph.TupleList(GV,directed=flag,vertex_name_attr='name')  ## 构建图
    return  g

def getStats(g):
    '''diameter,density,cliqu_number,avg_path_length,vcount,coef,assortativity,omega,mean_weights'''
    return  [g.diameter(),g.density(),g.clique_number(),g.average_path_length(),\
            g.vcount(),\
            g.transitivity_undirected(),## 聚集系数
            g.assortativity_degree(directed=False)  ##同配系数
#             g.omega()
#             np.array(g.es['weight']).mean()
            ]  
        

        
def GetCommStats(rel_net):
    g=getGraph(rel_net)
    names=g.vs['name']
    weights=g.es['weight']
    cluster=IGraph.community_label_propagation(g,weights='weight').as_cover()
    if data.shape[1]==3:
        pg_df=pd.DataFrame(g.pagerank(weights='weight'),index=names,columns=['pg'])
    else:
        pg_df=pd.DataFrame(g.pagerank(weights='weight'),index=names,columns=['pg'])

    cluster=[line for line in  list(cluster)  if line>6]

    columns=['diameter','density','clique_number','average_path_length','nums','coef','pg_mean']
    comm_stats=[]
    for line in cluster:
        pg_mean=pg_df.iloc[line].mean().values[0]
        gs=getStats(g.subgraph(line))+[pg_mean]
        comm_stats.append(gs)
    x=pd.DataFrame(comm_stats,columns=columns)
    return x

def getClusterStats(g,cluster):
    cluster=[line for line in  list(cluster)  if line>6]

    columns=['diameter','density','clique_number','average_path_length','nums','coef']
    comm_stats=[]
    for line in cluster:
        pg_mean=pg_df.iloc[line].mean().values[0]
        gs=getStats(g.subgraph(line))+[pg_mean]
        comm_stats.append(gs)
    x=pd.DataFrame(comm_stats,columns=columns)
    return x

from multiprocessing  import Pool
import igraph  as ig 
def getCommStats(comm):
    '''获取每个社区的特性'''
    if isinstance(sms_com,ig.clustering.VertexClustering):
        comm=comm.subgraphs()
    pool=Pool(8)
    sub_stats=pool.map(getStats,comm)
    pool.close()
    pool.join()
    columns='diameter,density,cliqu_number,avg_path_length,vcount,coef,assortativity,omega,mean_rel'.split(',')
#     sub_stats=map(getStats,rel_comm_lpa.subgraphs())
    sub_stats=pd.DataFrame(sub_stats,columns=columns)
    return sub_stats
def GetGVlues(fname):
    df=pd.read_csv(fname,index_col=0)
    df=df[df.cust_a!=df.cust_b]
    print df.shape[0]
    g=getGraph(df.values,flag=True)
    return g

In [ ]:
call_record=pd.read_csv('chapter1data/net_rel.csv',index_col=0)
call_record=call_record[call_record.custorm_id!=call_record.cust_b]
g=getGraph(call_record.values)


In [ ]:
app_conatcts=getGraph('chapter1data/app_contacts.csv',U=False)
sms=getGraph('chapter1data/sms_net.csv',U=False)
# call_record=getGraph('chapter1data/call_record_net.csv',U=False)

In [175]:
MPC=GetGVlues('chapter1data/app_contacts.csv')
CDR=GetGVlues('chapter1data/call_record_net.csv')
SMS=GetGVlues('chapter1data/sms_net.csv')

18520891
11391669
5209265


##   全局结构特性计算

In [160]:
Gdict={'MPC':MPC,'CDR':CDR,'SMS':SMS}

In [3]:
MPC_clu=MPC.community_label_propagation()
# CDR_clu=CDR.community_label_propagation()
SMS_clu=SMS.community_label_propagation()

In [148]:
Gdict.iteritems()

<dictionary-itemiterator at 0x7fffabc10158>

In [149]:
for x,y in Gdict.iteritems():
    print  x,y.assortativity_degree()

SMS -0.00777656392003
MPC 0.00297657340001
CDR -0.00204483891518


In [161]:
for x,y in Gdict.iteritems():
    print  x,y.assortativity_degree()

SMS 0.0255365659244
MPC 0.0053106797213
CDR -0.0216421709658


In [169]:
del_node=lambda  G:G.vs.select(_degree_gt=500).delete()
del_node(MPC)

In [172]:
for x,y in Gdict.iteritems():
    print  x,y.assortativity_degree(False)

SMS 0.0516431357806
MPC 0.130167054406
CDR 0.0112964616639


In [171]:
for x,y in Gdict.iteritems():
    print  x,y.assortativity_degree()

SMS 0.2351857565
MPC 0.221991455306
CDR 0.0161989974145


In [8]:
SMS=pd.read_csv('chapter1data/call_record_net.csv',index_col=0)
SMS.head(3)

,cust_a,cust_b
0,1000498,154011
1,1000643,2028579
2,1000672,84810


In [28]:
mpc_stats=getStats(MPC_clu.subgraph(67))

In [33]:
getStats?  ##diameter,density,cliqu_number,avg_path_length,vcount,coef,assortativity,omega

In [32]:
mpc_stats

[16,
 8.270481462795377e-05,
 10,
 6.189767372605167,
 64361,
 0.04304632549945,
 0.041758109742633075,
 10]

In [31]:
sms_stats=getStats(SMS_clu.giant())

In [53]:
x=SMS.components(1).giant()

In [62]:
getStats(SMS_clu.subgraph(11))

[6,
 0.15644753476611886,
 6,
 2.1338495575221237,
 113,
 0.1638215599358672,
 -0.48049984024075676,
 6]

In [34]:
SMS_clu

[11,
 0.0003097094195711242,
 4,
 2.8800974411042115,
 8267,
 0.00015083629241746988,
 -0.32951557038214235,
 4]

In [40]:
SMS.assortativity_degree()

-0.00777656392003368

In [41]:
MPC.assortativity_degree()

0.002976573400010509

In [42]:
CDR.assortativity_degree()

-0.0020448389151849354

In [64]:
cc=SMS_clu.giant()

In [66]:
cc.maxdegree()

5205

In [68]:
x=pd.DataFrame(cc.degree())

In [163]:
ss=MPC.vs.select(_degree_gt=500)

In [164]:
ss.delete?

In [109]:
ss=CDR.vs.select(_degree_gt=500)

In [123]:
CDR.assortativity?

In [127]:
MPC.assortativity()

In [140]:
g=x.as_directed()

In [143]:
g.assortativity_degree(True)

0.009582971860261774

In [112]:
x=CDR.components().giant()

In [113]:
CDR.assortativity_degree()

0.011296461663930715

In [142]:
x.assortativity_degree()

0.009582971860261733

In [81]:
SMS.vcount()

1527219

### 中观结构计算

In [ ]:
def getStats(g):
    '''diameter,density,cliqu_number,avg_path_length,vcount,coef,assortativity,omega,mean_weights'''
    return  [g.diameter(),g.density(),g.clique_number(),g.average_path_length(),
            g.transitivity_undirected(),## 聚集系数
            g.assortativity_degree(directed=False)  ##同配系数
            ]  
        
    
def getCommStats(comm):
    '''获取每个社区的特性'''
    if isinstance(sms_com,ig.clustering.VertexClustering):
        comm=comm.subgraphs()
    pool=Pool(8)
    sub_stats=pool.map(getStats,comm)
    pool.close()
    pool.join()
    columns='diameter,density,cliqu_number,avg_path_length,vcount,coef,assortativity,omega,mean_rel'.split(',')
#     sub_stats=map(getStats,rel_comm_lpa.subgraphs())
    sub_stats=pd.DataFrame(sub_stats,columns=columns)
    return sub_stats

In [194]:
sms_com=SMS.as_undirected().components()

In [213]:
sms_coms=sms_com.subgraphs()

KeyboardInterrupt: 

In [208]:
sms_coms

True

In [195]:
ss=pd.DataFrame(sms_com.sizes())

In [196]:
ss[0].max()

1012668

In [214]:
d=ss[(ss[0]>5)&(ss[0]<ss[0].max())]

In [216]:
d.shape

(12335, 1)